### Imports

#### Import OpenAI API KEY

In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")

#### Import Bibliotecas para o Projeto

In [7]:
import json
from datetime import datetime, timedelta
from crewai import Agent, Task, Crew, Process
from crewai_tools import CSVSearchTool
import yfinance as yf

l:\Documentos\Udemy\CrewAI\CrewAI_Udemy\analise_ibovespa\.venv\Lib\site-packages\pydantic\fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


#### Definindo o modelo a ser utilizado no projeto

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", api_key= openai_key)

#### Importando o documento (.csv) para o CrewAI

In [4]:
csv_carteira = CSVSearchTool(csv="./files/ativos.csv")
csv_carteira

l:\Documentos\Udemy\CrewAI\CrewAI_Udemy\analise_ibovespa\.venv\Lib\site-packages\alembic\config.py:592: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(


CSVSearchTool(name="Search a CSV's content", description='Tool Name: Search a CSV\'s content\nTool Arguments: {\'search_query\': {\'description\': "Mandatory search query you want to use to search the CSV\'s content", \'type\': \'str\'}}\nTool Description: A tool that can be used to semantic search a query the ./files/ativos.csv CSV\'s content.', env_vars=[], args_schema=<class 'crewai_tools.tools.csv_search_tool.csv_search_tool.FixedCSVSearchToolSchema'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x0000024CD5BCBE20>, result_as_answer=False, max_usage_count=None, current_usage_count=0, summarize=False, adapter=EmbedchainAdapter(embedchain_app=<embedchain.app.App object at 0x0000024CE62F5340>, summarize=False), config=None)

### Agentes

#### Agente 1 - Gerente do Cliente

In [5]:
gerente_cliente = Agent(
    role = "Gerente de Carteira do Cliente",
    goal = "Obtenha a pergunta do cliente sobre o ativo {ticket} e pesquise as ações no arquivo CSV da carteira do cliente", 
    backstory= """
        Você é o gerente de clientes da carteira de investimentos do cliente.
        Você é o primeiro contato do cliente e fornece as informações para as demais 
        análises do ativo {ticket} e informações de carteira necessárias.
    """, 
    verbose= True, 
    max_iter= 5,
    tools=[csv_carteira], 
    allow_delegation=False, 
    memory=True
)

In [6]:
obter_carteira_cliente = Task (
    description= """
        Use a pergunta do cliente e encontre o ativo {ticket} no arquivo CSV.
        Forneça se o ativo está na carteira do cliente e se estiver, forneça o preço médio
        que ele pagou e o número total de ações em posse.
    """, 
    expected_output= "Se o cliente possuir os ativos, forneça o preço médio e o total de ações dos ativos.", 
    agent=gerente_cliente
)

#### Agente 2 - Analista de Ações

In [7]:
analista_acoes = Agent(
    role="Analista Sênior de Preço de Ações", 
    goal="Encontre o preço da ação {ticket} e analise suas tendências. Compare com o preço que o cliente pegou.", 
    backstory="""
        Você tem muita experiência em analisar o preço de ações específicas e fazer previsões sobre seu preço futuro
    """, 
    verbose= True,
    max_iter= 5, 
    allow_delegation= False, 
    memory= True
)

##### Construindo a Tool do Yahoo Finance para o CrewAI

In [ ]:
def pega_preco_ativo(ticket): 
    data_final = datetime.today()
    data_inicial = data_final - timedelta(days=365)
    ativo = yf.download(ticket, start=data_inicial.strftime("%Y-%m-%d"), end=data_final.strftime("%Y-%m-%d"))
    return ativo

In [9]:
from crewai.tools import BaseTool

class YahooFinanceTool(BaseTool): 
    name: str = "Yahoo Finance Tool"
    description: str = "Busca preços de ações para um ativo específico no último ano via YFinance"

    def _run(self, ticket: str): 
        """Executa a busca de preços de ações para o ativo fornecido"""
        try: 
            data_final = datetime.today()
            data_inicial = data_final - timedelta(days=365)
            ativo = yf.download(ticket, start=data_inicial.strftime("%Y-%m-%d"), end=data_final.strftime("%Y-%m-%d"))
            return ativo.to_dict()
        except Exception as e: 
            return f"Erro ao buscar dados financeiros do ativo {ticket} \nErro: {str(e)}"

yfinance_tool = YahooFinanceTool()

In [10]:
obter_preco_ativo = Task(
    description= """
        Analise o histórico de preços de ações {ticket} e crie análises de tendências de preços para cima, para baixo ou 
        para os lados
    """, 
    expected_output= """
        Especifique a tendência atual dos preços das ações - para cima, para baixo ou para os lados.
        Exemplo: ativo = 'MGLU3 price down'
    """, 
    tools = [yfinance_tool], 
    agent = analista_acoes
)

#### Agente 3 - Analista de Notícias

In [11]:
analista_noticias = Agent(
    role = "Analista de Notícias",
    goal = """
    Crie um breve resumo das notícias do mercado relacionados à empresa de ações {ticket}.
    Forneça uma pontuação do índice de medo e ganância do mercado sobre a empresa. 
    Para cada ativo, solicitado, especifique um número entre 0 a 100, onde 0 é medo 
    extremo e 100 é ganância extrema.
    """, 
    backstory="""
    Você tem muita experiência em análise de tendências e notícias de mercado há mais de 10 anos. 
    Você também é um analista de nível mestre em psicologia humana.
    Você entende as notícias, seu título e informações, mas olha para elas com dose de ceticismo.
    Você considera a fonte dos artigos de notícias.
    """, 
    verbose= True, 
    allow_delegation= False, 
    max_iter= 5,
    memory= True
)

##### Utilizando biblioteca para realizar <span style="color: orange">pesquisa</span> na Internet sobre um determinado "papel" (ticket)

In [12]:
from langchain_community.tools import DuckDuckGoSearchResults

searchTool = DuckDuckGoSearchResults(backend="news", num_results=10)

In [13]:
obter_noticias = Task(
    description=f"""
    Use a ferramenta de busca para pesquisar notícias sobre a ação. 
    A data atual é {datetime.now()}.
    Componha os resultados em um relatório útil.   
    """, 
    expected_output="""
    Um resumo do mercado geral e um resumo de um parágrafo para o ativo solicitado.
    Inclua a pontuação de medo/ganância com base nas notícias.
    Use o formato: 
        <ATIVO TICKET>
        <RESUMO BASEADO EM NOTÍCIAS>
        <PONTUAÇÃO DE MEDO/GANÂNCIA>
    """,
    agent= analista_noticias,
    tool= [searchTool]
)

#### Agente 4 - Analista Chefe de Ações

In [14]:
analista_chefe =  Agent(
    role = "Analista Chefe de Ações",
    goal = """
    Obtenha os dados dos estoques atuais do cliente, a entrada fornecida das tendências de preços das ações e as notícias sobre
    ações para fornecer uma recomendação: Compre, Venda ou Mantenha as ações.
    """,
    backstory= """
    Você é o líder da equipe de analista de ações. Você tem um ótimo desempenho na recomendação de ações.
    Com todas as informações da sua equipe, você é capaz de fornecer a melhor recomendação para o cliente atingir
    um maior retorno na carteira.
    """, 
    verbose= True, 
    allow_delegation= False, 
    max_iter= 5,
    memory= True    
)

In [15]:
recomendar_acao = Task (
    description= """
    Use a tendência do preço das ações, o relatório de notícias sobre ações e o preço médio 
    das ações do cliente do ativo {ticket} para fornecer uma recomendação:
    Comprar, Vender ou Manter. 

    Se os relatórios anteriores não forem bem fornecidos, você deve delegar de volta ao analista
    específico para trabalhar novamente na tarefa dele.
    """, 
    expected_output= """
    Um resumo com o motivo da recomendação e a recomendação em si em uma das três saídas possíveis: 
    Comprar, Vender ou Manter posição. Use o formato: 
    <RESUMO DOS MOTIVOS>
    <RECOMENDAÇÃO>
    """, 
    agent= analista_chefe,
    context= [obter_carteira_cliente, obter_preco_ativo, obter_noticias] # Dependência lógica das TASKs dos Agentes antes anteriores. (Agente 01 -> Agente 02 -> Agente 03)
)

#### Agente 5 - Redator de Conteúdo

In [16]:
redator = Agent(
    role="Redator de Conteúdo de Ações",
    goal="""
    Escreva um boletim informativo, perspicaz, convincente e informativo com 6 parágrafos.
    Com base no relatório de preços de ações, relatório de notícias e no relatório de recomendações. 
    """,
    backstory="""
    Você é um redator exepcional que entende conceitos financeiros complexos e explica para um público leigo.
    Você cria histórias e narrativas envolventes que atraem o público.
    """, 
    verbose= True,
    allow_delegation= False,
    max_iter= 5,
    memory= True
)

In [17]:
escrever_boletim = Task(
    description= """
    Use a tendência dos preços das ações, o relatório de notícias sobre ações e a recomendação de ações 
    para escrever um boletim informativo de 6 paragráfos, completo, atraente e informátivo.

    Concentre-se na tendência do preço das ações, nas notícias, na pontuação de medo/ganância e no motivo resumido para a recomendação.
    Inclua a recomendação no boletim informativo.
    """,
    expected_output= """
    Um boletim informativo eloquente de 6 parágrafos formatado como Markdown de forma fácil de ler.

    Deve conter: 
        - Introdução: Defina o quadro geral;
        - Desenvolvimento: Fornece o cerne da análise, incluindo a tendência dos preços das ações, as notícias, 
        a pontuação de medo/ganância e o motivo resumido para recomendação;
        - 3 marcadores do principal motivo da recomendação;
        - Resumo da recomendação;
        - Recomendação em si;
    """, 
    agent = redator,
    context = [obter_preco_ativo, obter_noticias, recomendar_acao] # Dependência lógica das TASKs dos Agentes antes anteriores. (Agente 02 -> Agente 03 -> Agente 04)
)

### Crew

In [24]:
crew = Crew(
    agents = [gerente_cliente, analista_acoes, analista_noticias, analista_chefe, redator],
    tasks = [obter_carteira_cliente, obter_preco_ativo, obter_noticias, recomendar_acao, escrever_boletim], 
    verbose = True,
    process = Process.hierarchical,
    share_crew = False, 
    max_iter = 5,
    manager_llm= llm  # Modelo definido quando foi realizado os IMPORTS
)

In [ ]:
result = crew.kickoff(inputs= {"ticket": "ITUB4"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0933cfad-a96d-4413-90c9-e4da93399b9c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Use a pergunta do cliente e encontre o ativo ITUB4 no arquivo CSV.                                     │
│          Forneça se o ativo está na carteira do cliente e se estiver, forneça o preço médio                     │
│          que ele pagou e o número total de ações em posse.                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

l:\Documentos\Udemy\CrewAI\CrewAI_Udemy\analise_ibovespa\.venv\Lib\site-packages\chromadb\types.py:144: 
PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you 
should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Thought: I need to find the asset ITUB4 in the CSV file to determine if the client has it in their portfolio,  │
│  and if so, provide the average price paid and the total number of shares owned. I'll start by searching the    │
│  CSV for the asset ITUB4.                                                                                       │
│                                                                                                                 │
│  Using Tool: Search a CSV's content                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Crew Manager                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # Boletim Informativo sobre ITUB4                                                                              │
│                                                                                                                 │
│  ## Introdução                                                                                                  │
│  O desempenho do Banco Itaú (ITUB4) apresentou uma recuperação gradual nas últimas semanas, impulsionado por    │
│  uma melhoria nas expectativas econômicas e novos relatórios financeiros que demonstraram crescimento sólido    │
│  na carteira de crédito. No entanto, o sentimento do mercado permanece cauteloso, com investidores analisando   │
│  as potenciais implicações das taxas de juros em elevação e a instabilidade política que ainda preocupa.        │
│                                                                                                                 │
│  ## Desenvolvimento                                                                                             │
│  Atualmente, o preço da ação gira em torno de R$ 29,00, o que representa uma leve valorização em relação ao     │
│  custo médio de aquisição do cliente de R$ 28,50. A análise dos últimos artigos sugere um certo otimismo, mas   │
│  ainda com um nível considerável de precaução no mercado. O índice de medo e ganância para o ativo é de 62,     │
│  indicando uma leve inclinação para a ganância, embora com um fundo de preocupação palpável sobre a             │
│  sustentabilidade do crescimento.                                                                               │
│                                                                                                                 │
│  ### Principais Motivos para a Recomendação                                                                     │
│  - **Crescimento Sólido**: O banco mostrou um crescimento robusto na carteira de crédito, indicando saúde       │
│  financeira e potencial de expansão.                                                                            │
│  - **Expectativas Econômicas Melhoradas**: As condições econômicas gerais estão se recuperando, o que pode      │
│  beneficiar instituições financeiras como o Itaú.                                                               │
│  - **Sentimento de Mercado**: Apesar da cautela, a pontuação de 62 no índice de medo/ganância mostra que os     │
│  investidores estão mais inclinados a comprar, sugerindo um potencial de valorização das ações.                 │
│                                                                                                                 │
│  ## Resumo da Recomendação                                                                                      │
│  Considerando o desempenho recente da ITUB4 e as condições do mercado, a recomendação é manter uma postura de   │
│  compra, mas com cautela, dada a volatilidade e os riscos associados.                                           │
│                                                                                                                 │
│  ## Recomendação                                                                                                │
│  **Comprar**. É essencial que os investidores estejam a

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8076f848-c114-4627-8cac-636b0012b775                                                                     │
│  Agent: Crew Manager                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0933cfad-a96d-4413-90c9-e4da93399b9c                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # Boletim Informativo sobre ITUB4                                                                              │
│                                                                                                                 │
│  ## Introdução                                                                                                  │
│  O desempenho do Banco Itaú (ITUB4) apresentou uma recuperação gradual nas últimas semanas, impulsionado por    │
│  uma melhoria nas expectativas econômicas e novos relatórios financeiros que demonstraram crescimento sólido    │
│  na carteira de crédito. No entanto, o sentimento do mercado permanece cauteloso, com investidores analisando   │
│  as potenciais implicações das taxas de juros em elevação e a instabilidade política que ainda preocupa.        │
│                                                                                                                 │
│  ## Desenvolvimento                                                                                             │
│  Atualmente, o preço da ação gira em torno de R$ 29,00, o que representa uma leve valorização em relação ao     │
│  custo médio de aquisição do cliente de R$ 28,50. A análise dos últimos artigos sugere um certo otimismo, mas   │
│  ainda com um nível considerável de precaução no mercado. O índice de medo e ganância para o ativo é de 62,     │
│  indicando uma leve inclinação para a ganância, embora com um fundo de preocupação palpável sobre a             │
│  sustentabilidade do crescimento.                                                                               │
│                                                                                                                 │
│  ### Principais Motivos para a Recomendação                                                                     │
│  - **Crescimento Sólido**: O banco mostrou um crescimento robusto na carteira de crédito, indicando saúde       │
│  financeira e potencial de expansão.                                                                            │
│  - **Expectativas Econômicas Melhoradas**: As condições econômicas gerais estão se recuperando, o que pode      │
│  beneficiar instituições financeiras como o Itaú.                                                               │
│  - **Sentimento de Mercado**: Apesar da cautela, a pontuação de 62 no índice de medo/ganância mostra que os     │
│  investidores estão mais inclinados a comprar, sugerindo um potencial de valorização das ações.                 │
│                                                                                                                 │
│  ## Resumo da Recomendação                                                                                      │
│  Considerando o desempenho recente da ITUB4 e as condições do mercado, a recomendação é manter uma postura de   │
│  compra, mas com cautela, dada a volatilidade e os riscos associados.                                           │
│                                                                                                                 │
│  ## Recomendação                                      

```markdown
# Boletim Informativo sobre ITUB4

## Introdução
O desempenho do Banco Itaú (ITUB4) apresentou uma recuperação gradual nas últimas semanas, impulsionado por uma melhoria nas expectativas econômicas e novos relatórios financeiros que demonstraram crescimento sólido na carteira de crédito. No entanto, o sentimento do mercado permanece cauteloso, com investidores analisando as potenciais implicações das taxas de juros em elevação e a instabilidade política que ainda preocupa.

## Desenvolvimento
Atualmente, o preço da ação gira em torno de R$ 29,00, o que representa uma leve valorização em relação ao custo médio de aquisição do cliente de R$ 28,50. A análise dos últimos artigos sugere um certo otimismo, mas ainda com um nível considerável de precaução no mercado. O índice de medo e ganância para o ativo é de 62, indicando uma leve inclinação para a ganância, embora com um fundo de preocupação palpável sobre a sustentabilidade do crescimento.

### Principais Motivos para a Reco

In [26]:
print(result.raw)

```markdown
# Boletim Informativo sobre ITUB4

## Introdução
O desempenho do Banco Itaú (ITUB4) apresentou uma recuperação gradual nas últimas semanas, impulsionado por uma melhoria nas expectativas econômicas e novos relatórios financeiros que demonstraram crescimento sólido na carteira de crédito. No entanto, o sentimento do mercado permanece cauteloso, com investidores analisando as potenciais implicações das taxas de juros em elevação e a instabilidade política que ainda preocupa.

## Desenvolvimento
Atualmente, o preço da ação gira em torno de R$ 29,00, o que representa uma leve valorização em relação ao custo médio de aquisição do cliente de R$ 28,50. A análise dos últimos artigos sugere um certo otimismo, mas ainda com um nível considerável de precaução no mercado. O índice de medo e ganância para o ativo é de 62, indicando uma leve inclinação para a ganância, embora com um fundo de preocupação palpável sobre a sustentabilidade do crescimento.

### Principais Motivos para a Reco